# Choose omni env

In [1]:
from pathlib import Path
import requests
import rasterio as rio
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

from omnicloudmask import predict_from_array, predict_from_load_func, load_multiband
import omnicloudmask

from functools import partial

import glob

/home/mmacander/miniconda3/envs/omni/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
omnicloudmask.__version__

'1.0.7'

Go to https://www.maxar.com/open-data/ scroll to the bottom, select one of the captured events (from 2023 or late 2022) and click 'Learn more', on the new page select one of the scenes and click the corresponding 'QUAD KEY' button, from the new popup right click on the 'Multispectral Image' button and select copy url. Past that url below to download and run inference.


In [3]:
url = "https://maxar-opendata.s3.us-west-2.amazonaws.com/events/Emilia-Romagna-Italy-flooding-may23/ard/32/120000303231/2023-05-23/1050010033C95B00-ms.tif"
# url = "https://maxar-opendata.s3.us-west-2.amazonaws.com/events/tonga-volcano21/ard/01/300222121030/2022-01-18/1040010071C26C00-ms.tif"
# url = "https://maxar-opendata.s3.us-west-2.amazonaws.com/events/Kalehe-DRC-Flooding-5-8-23/ard/35/300000303201/2023-05-12/1040010084614200-ms.tif"
# url = "https://maxar-opendata.s3.us-west-2.amazonaws.com/events/New-Zealand-Flooding23/ard/60/213311212303/2023-02-08/10300100E23B4300-ms.tif"
# url = "https://maxar-opendata.s3.us-west-2.amazonaws.com/events/Hurricane-Ian-9-26-2022/ard/17/031331321021/2022-10-01/104001007C700900-ms.tif"

Download the Maxar image for inference


In [4]:
file_name = "_".join(url.split("/")[-3:])
scene_path = Path.cwd() / f"Maxar data/{file_name}"
scene_path.parent.mkdir(exist_ok=True, parents=True)

if not scene_path.exists():
    print(f"Downloading scene from {url} \nto {scene_path}")
    with requests.get(url, stream=True) as response:
        with open(scene_path, "wb") as f:
            pbar = tqdm(
                unit="B", unit_scale=True, total=int(response.headers["Content-Length"])
            )
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                pbar.update(8192)
    pbar.close()
else:
    print(f"Scene already exists: {scene_path}")

Scene already exists: /import/sportello/shared/pchome/mmacander/programming/abr/akveg-map/01a_data_reflectance_vhr/Maxar data/120000303231_2023-05-23_1050010033C95B00-ms.tif


In [ ]:
scene_path

In [ ]:
scene_path=Path('/data/gis/raster_base/Alaska/AKVegMap/EVWHS/navy_north_slope/ortho_toa_subimages/20JUL14220836-M1BS-050300602010_01_P002_u16rf3338.tif')

In [ ]:
with rio.open(scene_path) as src:
    print(f"Original NoData Value: {src.nodata}")  # Prints existing NoData value


Maxar images are 50cm resolution so we will resample them to 10 m for inference


In [ ]:
required_res = 10

Some Maxar MS images have 4 channels others have 8, we need to work out which type we have and extract the Red Green (Blue for visualisation) and NIR bands.


In [ ]:
# src = rio.open(scene_path)
# channel_count = src.meta["count"]
# if channel_count == 4:
#     print("Found 4 channels, assuming RGBN")
#     GRB_NIR_bands = [1, 2, 3, 4]
# if channel_count == 8:
#     print("Found all 8 channels")
#     GRB_NIR_bands = [5, 3, 2, 8]

In [ ]:
src = rio.open(scene_path)
channel_count = src.meta["count"]
if channel_count == 4:
    print("Found 4 channels, assuming BGRN")
    BGR_NIR_bands = [1, 2, 3, 4]
if channel_count == 8:
    print("Found all 8 channels")
    BGR_NIR_bands = [3, 3, 5, 7]

Open the RGB+NIR bands at the required resolution


In [ ]:
# res = src.res
# RGBN_array = src.read(
#     GRB_NIR_bands,
#     out_shape=(
#         4,
#         int(src.height * res[0] / required_res),
#         int(src.width * res[0] / required_res),
#     ),
#     masked=True
# )

In [ ]:
res = src.res
BGRN_array = src.read(
    BGR_NIR_bands,
    out_shape=(
        4,
        int(src.height * res[0] / required_res),
        int(src.width * res[0] / required_res),
    ),
    masked=True
)

In [ ]:
BGRN_array

Separate the RGB bands for visualisation and the RG+NIR bands for inference.


In [ ]:
# RGB_array = BGRN_array[:3]
RGB_array = BGRN_array[[2,1,0]]
RGB_array = RGB_array.transpose(1, 2, 0)

RGN_array = BGRN_array[[2, 1, 3]]
RGN_array_nd0 = np.where(RGN_array.mask, 0, RGN_array.data)

Normalise the RGB channels and plot the image


In [ ]:
RGB_array_norm = np.power(RGB_array.astype("float32"), 0.5)
RGB_array_norm = RGB_array_norm - np.min(RGB_array_norm)
RGB_array_norm = RGB_array_norm / np.max(RGB_array_norm)

RGB_array_norm = np.clip(RGB_array_norm, 0, 1)
plt.imshow(RGB_array_norm)

Perform inference on the RG+NIR bands using the 'predict_from_array' function


In [ ]:
mask = predict_from_array(
    # RGN_array,
    RGN_array_nd0
)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 6))
cmap = ListedColormap(["black", "white", "white", "grey"])

ax[0].imshow(RGB_array_norm)
ax[1].imshow(RGB_array_norm)
ax[1].contour(mask[0] > 0, colors="red", linewidths=1.5)
ax[1].contour(mask[0] == 3, colors="red", linewidths=1.5)
ax[2].imshow(mask[0], vmin=0, vmax=3, cmap=cmap, interpolation="nearest")

legend_labels = ["Clear", "Shadow", "Cloud"]
legend_colors = ["black", "grey", "white"]
patches = [
    mpatches.Patch(
        facecolor=legend_colors[i], edgecolor="black", label=legend_labels[i]
    )
    for i in range(3)
]
fig.legend(handles=patches, loc="lower right", bbox_to_anchor=(0.904, 0.05))

for a in ax:
    a.set_xticks([])
    a.set_yticks([])

ax[0].set_title("True colour")
ax[1].set_title("True colour with OCM\nprediction contours")
ax[2].set_title("OCM prediction")

plt.show()

In [ ]:
mask

In [ ]:
RGN_array

In [5]:
load_maxar_pgc_4band_10m = partial(load_multiband, resample_res=10.0, band_order=[3,2,4])


In [45]:
# ortho_subimage_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/navy_north_slope/ortho_toa_subimages"
# ortho_subimage_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/nome_beaver/ortho_toa_subimages"
ortho_subimage_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages"

# ortho_subimage_cloud_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/navy_north_slope/ortho_toa_subimages_cloud"
# ortho_subimage_cloud_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/nome_beaver/ortho_toa_subimages_cloud"
ortho_subimage_cloud_dir = "/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages_cloud"

Path(ortho_subimage_cloud_dir).mkdir(parents=True, exist_ok=True)

scene_paths = glob.glob(ortho_subimage_dir + "/*.tif")
print(scene_paths[:10])


['/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages/23JUL21210509-M1BS-050296456010_01_P008_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages/23JUL21205720-M1BS-050296474010_01_P010_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages/21AUG30204810-M1BS-050296461010_01_P005_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages/23JUL26212327-M1BS-050296496010_01_P005_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages/24JUN29212914-M1BS-050296500010_01_P008_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages/23JUL21210510-M1BS-050296456010_01_P007_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages/24JUN29212846-M1BS-050296485010_01_P001_u16rf3338.tif', '/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages/13JUN18214810-M

In [46]:
import torch
torch.get_default_device()

for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i))#.name)


_CudaDeviceProperties(name='NVIDIA GeForce RTX 4090', major=8, minor=9, total_memory=24217MB, multi_processor_count=128, uuid=295cbb7d-a5d5-7176-4292-e190315a0713, L2_cache_size=72MB)


In [47]:
torch.set_default_device('cuda')
torch.get_default_device()


device(type='cuda', index=0)

In [48]:
# scene_path=scene_paths[4]

In [49]:
for scene_path in scene_paths:
    pred_path = predict_from_load_func(
        # scene_paths=scene_paths,
        scene_paths=[scene_path],
        load_func=load_maxar_pgc_4band_10m,
        inference_dtype="bf16",
        no_data_value=65535,
        output_dir=ortho_subimage_cloud_dir,
        inference_device='cuda',
        mosaic_device='cpu',
        overwrite=False,
        batch_size=1,
        # patch_size=256,
        # patch_overlap=128
    )
    print(f"Predicted {pred_path}")
    
    with torch.no_grad():
        torch.cuda.empty_cache()



Running inference using cuda bfloat16: 100%|██████████| 1/1 [00:05<00:00,  5.77s/it]


Predicted [Path('/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages_cloud/23JUL21210509-M1BS-050296456010_01_P008_u16rf3338_OCM_v1_0_7.tif')]



/home/mmacander/miniconda3/envs/omni/lib/python3.11/site-packages/omnicloudmask/cloud_mask.py:145: UserWarning: Significant no-data areas detected. Adjusting patch size to 266px and overlap to 133px to minimize no-data patches.
  warnings.warn(

Running inference using cuda bfloat16: 100%|██████████| 1/1 [00:04<00:00,  4.32s/it]


Predicted [Path('/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages_cloud/23JUL21205720-M1BS-050296474010_01_P010_u16rf3338_OCM_v1_0_7.tif')]



Running inference using cuda bfloat16: 100%|██████████| 1/1 [00:07<00:00,  7.12s/it]


...

Predicted [Path('/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages_cloud/23AUG23205755-M1BS-050296473010_01_P009_u16rf3338_OCM_v1_0_7.tif')]



Running inference using cuda bfloat16: 100%|██████████| 1/1 [00:09<00:00,  9.06s/it]


Predicted [Path('/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages_cloud/23AUG23205748-M1BS-050296473010_01_P002_u16rf3338_OCM_v1_0_7.tif')]



/home/mmacander/miniconda3/envs/omni/lib/python3.11/site-packages/omnicloudmask/cloud_mask.py:145: UserWarning: Significant no-data areas detected. Adjusting patch size to 521px and overlap to 260px to minimize no-data patches.
  warnings.warn(

Running inference using cuda bfloat16: 100%|██████████| 1/1 [00:05<00:00,  5.03s/it]

Predicted [Path('/data/gis/raster_base/Alaska/AKVegMap/EVWHS/mentasta/ortho_toa_subimages_cloud/20AUG06212411-M1BS-050296492010_01_P003_u16rf3338_OCM_v1_0_7.tif')]


In [44]:
# To clear cache between runs
with torch.no_grad():
    torch.cuda.empty_cache()
